In [2]:
from gensim.models import Word2Vec

## 目录查看

In [23]:
!ls -alh stage1/output

total 6.2G
drwxr-xr-x 3 kesci root  4.0K Jun  7 14:37 .
drwxr-xr-x 5 kesci users 4.0K Jun  7 01:37 ..
-rw-r--r-- 1 kesci users 889M Jun  7 09:35 fastText_supervised.bin
drwxr-xr-x 2 kesci users 4.0K Jun  2 07:41 stats
-rw-r--r-- 1 kesci users 1.4G Jun  7 12:49 test_v1_feature_text.csv
-rw-r--r-- 1 kesci users 2.8G Jun  7 11:46 train_v1_feature_text.csv
-rw-r--r-- 1 kesci users  20M Jun  7 08:50 word2vec.model
-rw-r--r-- 1 kesci users 242M Jun  7 08:50 word2vec.model.trainables.syn1neg.npy
-rw-r--r-- 1 kesci users 242M Jun  7 08:50 word2vec.model.wv.vectors.npy
-rw-r--r-- 1 kesci users 794M Jun  7 14:39 word2vec.vec


In [6]:
# 将word2vec转化为可读取的vec格式
model = Word2Vec.load("stage1/output/word2vec.model")
model.wv.save("stage1/output/word2vec.kv")
# model.wv.save_word2vec_format("stage1/output/word2vec.vec", binary=False)
del model

/Users/niudong/workon_home/py3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/Users/niudong/workon_home/py3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [16]:
!ls -alh stage1/input

total 2.9G
drwxr-xr-x 2 kesci root  4.0K Jun  7 09:00 .
drwxr-xr-x 5 kesci users 4.0K Jun  7 01:37 ..
-rw-r--r-- 1 kesci users 852M Jun  7 09:00 fastText_labeled_content.txt
-rw-r--r-- 1 kesci users 426M Jun  7 05:34 test_copy.csv
-rw-r--r-- 1 kesci users 866M Jun  7 05:33 train_01_0607.csv
-rw-r--r-- 1 kesci users 747M Jun  7 06:11 word2vec_sentences.txt


## 结果提交

In [ ]:
import csv
query_ids = []
query_title_ids = []
predicts = []
def ProcessForFastText():
  with Timer("Predict with fastText"):
      with open('../input/bytedance/first-round/test.csv') as csv_file:
          csv_reader = csv.reader(csv_file, delimiter=',')
          line_count = 0
          for row in csv_reader:
              query_ids.append(row[0])
              query_title_ids.append(row[2])
              predicts.append(loaded_model.predict(row[1]+" "+row[3], k=2)[1][1])
              line_count+=1
              if line_count % 5000000 == 0: # 10000*10000 一亿条要一晚上
                  print(f'Processed {line_count} lines.')
ProcessForFastText()

In [ ]:
print(query_ids[:10])
print(query_title_ids[:10])
print(predicts[:10])

In [ ]:
import pandas as pd
res = pd.DataFrame({
    "query_id": query_ids,
    "query_title_id": query_title_ids,
    "click": predicts
})
res.to_csv("submit_fastText.csv", index=None, header=None)
! ls -al

In [ ]:
print(pd.read_csv("submit_fastText.csv")[:5])

In [ ]:
!./kesci_submit -token 490475a1ae106f67 -file submit_fastText.csv

In [ ]:
print(loaded_model.predict("1427 564550 2136 11 60847 9156 12497 11", k=2))
! ls -al